## Topographic Relative Position: Bathymetric Position Index (BPI)   
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 

### Initial setup

In [ ]:
import numpy as np
import rasterio
from scipy.ndimage import uniform_filter

### Define data path

In [ ]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
input_file_name = 'Test_DEM.tif'
#input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
output_file_name = 'Test_pyBPI.tif'
#output_file_name = 'Tokeland_pyBPI.tif'
#output_file_name = 'Nemah_pyBPI.tif'
#output_file_name = 'Francies_pyBPI.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [ ]:
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [ ]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

## Bathymetric Position Index (BPI)

The Bathymetric Position Index (BPI) measures the topographic position of a point relative to the surrounding sea floor. It identifies whether a point is in a topographic high, low, or a slope. BPI is calculated by comparing the elevation of a cell to the mean elevation of its surrounding cells within a specified neighborhood. The equation for BPI is:

$$
BPI = Z_{\text{cell}} - \text{mean}(Z_{\text{neighborhood}})
$$

where:
- $ Z_{\text{cell}} $ is the elevation of the cell.
- $ \text{mean}(Z_{\text{neighborhood}}) $ is the mean elevation of the surrounding neighborhood.


##### BPI function  

In [ ]:
def calculate_BPI(dem, window_size):
    """
    Calculate the Bathymetric Position Index (BPI) for a digital elevation model (DEM).
    
    :param dem: A 2D numpy array of the DEM.
    :param window_size: The size of the window to calculate the mean elevation (must be an odd number).
    :return: A 2D numpy array of the BPI.
    """
    # Calculate the mean elevation within the window
    mean_elevation = uniform_filter(dem, size=window_size, mode='nearest')
    
    # BPI is the difference between the cell elevation and the mean elevation
    BPI = dem - mean_elevation
    return BPI

In [ ]:
def calculate_BPI(dem, window_size):
    """
    Calculate the Bathymetric Position Index (BPI) for a digital elevation model (DEM).
    
    :param dem: A 2D numpy array of the DEM.
    :param window_size: The size of the window to calculate the mean elevation (must be an odd number).
    :return: A 2D numpy array of the BPI.
    """
    # Calculate the mean elevation within the window
    mean_elevation = uniform_filter(dem, size=window_size, mode='nearest')
    
    # BPI is the difference between the cell elevation and the mean elevation
    BPI = dem - mean_elevation
    return BPI

Calculate BPI with a given window

In [ ]:
# Calculate with a N x N window as an example
window_size = 15  # Replace with the desired window size

# Calculate BPI
BPI = calculate_BPI(dem, window_size)

Output data into GeoTIFF
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [ ]:
# Update metadata for output GeoTIFF
meta.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')

# Write BPI to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(BPI.astype(rasterio.float32), 1)